# Exercise - Text Mining - Classification - SCIKIT-LEARN

We will predict the category of a product based on the description of the product.

**The unit of analysis is a product**

In [137]:
import pandas as pd
import numpy as np

In [139]:
products = pd.read_csv('products.csv')

In [141]:
products.head(5)

,Category,Description
0,Electronics,HP 680 Original Ink Advantage Cartridge (Black...
1,Clothing & Accessories,Bold N Elegant Navy Blue Thin Summer Pregnancy...
2,Books,The Travel Book: A Journey Through Every Count...
3,Electronics,Tiny Deal Compact 10x25 Mini Binoculars Telesc...
4,Clothing & Accessories,Nimble House 16Pcs/Set Unisex Women Men No Tie...


In [143]:
products.shape

(5000, 2)

## Assign the "target" variable



In [146]:
target = products['Category']

## Assign the "text" (input) variable

In [149]:
input_data = products[['Description']]

## Split the data

In [152]:
from sklearn.model_selection import train_test_split

train_set, test_set, train_y, test_y = train_test_split(input_data, target, test_size=0.3, random_state=42)

In [154]:
train_set.shape, train_y.shape

((3500, 1), (3500,))

In [156]:
test_set.shape, test_y.shape

((1500, 1), (1500,))

# Data Prep

In [159]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

In [161]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

## Sklearn: Text preparation

Create a pipeline to transform the text column and create a term by document matrix.

In [164]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    # First, conver the dataframe column to a numpy array. Then, call the ravel function to make it one-dimensional
    return np.array(df).ravel()

In [166]:
new_col(train_set)

array(['Dehman 100-Percent Silky Satin Hair Beauty Pillowcase, Black, King Size DEHMANSATIN Silk originated in INDIA and it is an honor to present this ancient oriental gift to you. The collective wisdom of the sericulture people of this land has been condensed DEHMAN has always adhered to its original heart from generation to generation. From raw satin silk material collection to processing and to finished products,each step is carefully selected. DEHMAN hopes that every customer will have a wonderful satin silk experience and enjoy the silky noble life. Why Choose DEHMAN Pillowcase ? Material：DEHMAN Satin Silk pillowcase is crafted by 19 momme pure mulberry silk. Both sides of the pillowcase is organic and natural satin silk.Light weight and easy to carry. Design: It is designed with hidden zipper closure.With Queen(20x30inches) and King(20x36inches) and Standard size(20x26inches) in various kinds colours. Quality：Superior durable plain color ,not easy to run after washing.Exquisite 

In [168]:
text_column = ['Description']

In [170]:
number_svd_components = 300

In [172]:
text_transformer = Pipeline(steps=[
                ('my_new_column', FunctionTransformer(new_col)),
                ('text', TfidfVectorizer(stop_words='english')),
                ('svd', TruncatedSVD(n_components=number_svd_components, n_iter=10))
            ])

In [174]:
preprocessor = ColumnTransformer([
                     ('text', text_transformer, text_column),
                    ],
        remainder='drop')

#passtrough is an optional step. You don't have to use it.

In [176]:
#Fit and transform the train data
train_target = preprocessor.fit_transform(train_set)

train_target

array([[ 1.32127476e-01, -3.83815352e-02, -5.52127985e-02, ...,
         7.40545834e-03,  2.75921509e-02,  1.83845662e-02],
       [ 1.19672838e-03,  1.09071177e-03, -9.41522160e-06, ...,
        -5.70126073e-03, -3.00987407e-03,  4.43523891e-02],
       [ 5.74701078e-02,  5.16487292e-02,  8.81057288e-03, ...,
        -1.61783986e-02,  4.96770664e-03, -3.15268555e-02],
       ...,
       [ 1.59120292e-01, -7.66943792e-02, -1.60781830e-01, ...,
        -4.54302925e-03,  1.00522229e-02, -9.24976918e-03],
       [ 8.31440381e-02, -4.33959857e-02, -3.44149656e-02, ...,
         2.40267390e-02, -6.80252571e-03,  3.68320727e-02],
       [ 9.66538621e-02, -3.95970843e-02, -1.19857668e-01, ...,
         3.29236940e-02, -8.91255914e-04,  6.48951622e-04]])

In [177]:
train_target.shape

(3500, 300)

In [178]:
# Transform the test data
test_target = preprocessor.transform(test_set)

test_target

array([[ 1.97114994e-02,  1.21761976e-02,  5.64651570e-03, ...,
        -1.23239731e-02, -6.81197658e-03,  1.16464412e-02],
       [ 8.02758518e-02, -4.06088051e-02,  2.60699499e-02, ...,
        -1.30069508e-02,  1.24281867e-02, -1.55379411e-02],
       [ 1.73274443e-01, -7.02506391e-02, -4.60694990e-02, ...,
         1.89616925e-02,  1.24876826e-03, -1.61404956e-02],
       ...,
       [ 8.83440905e-02,  3.66329274e-02,  6.27166859e-03, ...,
        -7.69587690e-03, -3.83849736e-03, -4.42217778e-03],
       [ 8.82624542e-02, -3.47200813e-02,  3.56394126e-02, ...,
        -2.18407515e-02,  5.03488116e-03,  3.15663405e-02],
       [ 1.98262116e-02, -9.44310695e-03,  2.90959470e-02, ...,
        -1.14423286e-03, -1.14682098e-02, -1.49186272e-04]])

In [179]:
test_target.shape

(1500, 300)

# Calculate the baseline

In [181]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_target, train_y)

DummyClassifier(strategy='most_frequent')

In [182]:
from sklearn.metrics import accuracy_score

In [183]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_target)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

Baseline Train Accuracy: 0.38142857142857145


In [191]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_target)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

Baseline Test Accuracy: 0.386


# Try one of the classifiers we have covered so far

Stochastic Gradient Descent Classifier

In [194]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(max_iter=100)

In [196]:
sgd_clf.fit(train_target, train_y)

SGDClassifier(max_iter=100)

## Accuracy

In [199]:
#Train accuracy

train_y_pred = sgd_clf.predict(train_target)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9491428571428572


In [201]:
#Test accuracy

test_y_pred = sgd_clf.predict(test_target)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.94


## Generate the confusion matrix

In [204]:
from sklearn.metrics import confusion_matrix

#Usually created on test set
confusion_matrix(test_y, test_y_pred)

array([[331,   6,   5,  23],
       [  0, 252,   1,   5],
       [  7,   0, 270,  21],
       [  5,   7,  10, 557]])

# Try another classifier we have covered so far

Random Forest

In [207]:
from sklearn.ensemble import RandomForestClassifier 

from sklearn.metrics import accuracy_score

In [209]:
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1) 

rnd_clf.fit(train_target, train_y)

RandomForestClassifier(max_leaf_nodes=16, n_jobs=-1)

## Accuracy

In [212]:
from sklearn.metrics import accuracy_score

In [214]:
#Train accuracy

train_y_pred = rnd_clf.predict(train_target)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

Train acc: 0.9082857142857143


In [216]:
#Test accuracy

test_y_pred = rnd_clf.predict(test_target)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

Test acc: 0.8973333333333333


## Generate the confusion matrix

In [219]:
from sklearn.metrics import confusion_matrix

#Usually created on test set
confusion_matrix(test_y, test_y_pred)

array([[327,   6,   4,  28],
       [  1, 237,   4,  16],
       [ 11,   0, 231,  56],
       [ 10,   9,   9, 551]])

# Check for the number of components

**Determine whether increasing/decreasing the number of components increases/decreases the two models' accuracies** Discuss your findings below.

Between these two, the first classifier, Stochastic Gradient Descent Classifier showed a higher accuracy of .94. For that one the max iteration was 100, so increasing the number increases the accuracy. 